# Assumptions
    Checks if the graph and its input parameters are valid.

In [5]:
def check_graph(nodes, arcs, capacities, costs, supply, demand):
    """
    Arguments:
        nodes (list): List of nodes in the graph
        arcs (list): List of arcs (edges) in the graph
        capacities (dict): Dictionary containing the capacity of each arc
        costs (dict): Dictionary containing the cost of each arc
        supply (dict): Dictionary containing the supply value for supply nodes
        demand (dict): Dictionary containing the demand value for demand nodes
    Returns:
        bool: True if the graph and input parameters are valid, False otherwise
    """

    # Check if all values of capacities, costs, supply, and demand are integers
    for value_dict, value_name in [(capacities, 'capacities'), (costs, 'costs'), (supply, 'supply'), (demand, 'demand')]:
        if not all(isinstance(value, int) for value in value_dict.values()):
            print(f"Error: All values in {value_name} should be integers.")
            return False

    # Check if the sum of supplies is equal to the sum of demands
    if sum(supply.values()) != sum(demand.values()):
        print("Error: The sum of supplies should be equal to the sum of demands.")
        return False

# Successive Shortest Path
    While there is a Demand node:
        1. Select one supply node and find the shortest path between this node and other demand nodes.
        2. Augment flow along the shortest path from the selected supply node to demand nodes.

In [6]:
import math
import pprint
import random

def successive_shortest_path(nodes, arcs, capacities, costs, supply, demand):
    """
    The successive shortest path algorithm to solve the minimum cost flow problem.

    Arguments:
        nodes (list): List of nodes in the graph
        arcs (list): List of arcs (edges) in the graph
        capacities (dict): Dictionary containing the capacity of each arc
        costs (dict): Dictionary containing the cost of each arc
        supply (dict): Dictionary containing the supply value for supply nodes
        demand (dict): Dictionary containing the demand value for demand nodes

    Returns:
        Residual Graph
        dict: Dictionary containing the flow on each arc.
        Optimal Cost
    """

    # Check the assumptions
    result = check_graph(nodes, arcs, capacities, costs, supply, demand)

    if result is False:
        return  # Exit if the graph is not valid

    # Create residual graph
    residual_graph = {node: {} for node in nodes}
    for (u, v) in arcs:
        # Add edges to the residual graph with capacities, costs, and reduced costs
        residual_graph[u][v] = {'capacity': capacities[(u, v)], 'cost': costs[(u, v)], 'reduced_cost': costs[(u, v)]}

        # Add reverse edges with zero capacity and negated costs
        residual_graph[v][u] = {'capacity': 0, 'cost': -costs[(u, v)], 'reduced_cost': -costs[(u, v)]}

    # Initialize the flow on each arc and potentials for each node
    flow = {}
    for u, v in arcs:
        flow[(u, v)] = 0
        flow[(v, u)] = 0
    potential = {node: 0 for node in nodes}

    # Main loop (Performs while there is a demand node)
    while demand:
        # Select one supply node
        source = next(iter(supply))

        """
        Use Dijkstra's algorithm to find the shortest paths from a supply node to all nodes in the graph,
        1. Obtain predecessors to create the paths from selected source to all demand nodes.
        2. Use distance labels for updating the potentials of the nodes.
        """
        dist, prev = dijkstra(residual_graph, nodes, source)


        """
        Extract paths from the source node to all demand nodes
        by inserting the predecessors of nodes in a list until we reach the source node that has no predecessor.
        Then it will adds the list (path) to the dictionary so that we can track paths and their destination demands (dictionary keys)
        """
        paths = {}
        for demand_node in demand.keys():
                path = []
                node = demand_node
                while node is not None:
                    path.insert(0, node)
                    node = prev[node]
                paths[demand_node] = path

        if not paths:
            # No augmenting paths, terminate the algorithm
            break

        # make a list that contains all the keys from the demand dictionary
        target_demand_candidates = [target_demand for target_demand in demand.keys()]

        # While there is still some supply in the selected source and demand candidates list is not empty
        while supply[source] > 0 and target_demand_candidates:

             # Select one demand from candidates list
             target_demand = random.choice(target_demand_candidates)

             # Extract the edges along the path from the source to the selected demand node (target_demand)
             path_edges = zip(paths[target_demand], paths[target_demand][1:])

             # Check if all edges along the path have remaining capacity in the residual graph
             # if not, remove the selected demand from candidate list and go back and select another one
             if not all(residual_graph[u][v]['capacity'] > 0 for u, v in path_edges):
                    target_demand_candidates.remove(target_demand)
                    continue
             else:
                    # Find the minimum residual capacity along the path
                    min_capacity = min(residual_graph[u][v]['capacity'] for u, v in zip(paths[target_demand], paths[target_demand][1:]))

                    # Adjust the possible flow based on the minimum residual capacity along the path, supply and demand values
                    delta = min(min_capacity, supply[source], demand[target_demand])

                    # Update the flow and residual capacities along the path for the target_demand
                    for i in range(len(paths[target_demand]) - 1):
                            u, v = paths[target_demand][i], paths[target_demand][i + 1]
                            residual_graph[u][v]['capacity'] -= delta
                            residual_graph[v][u]['capacity'] += delta
                            flow[(u, v)] += delta
                            flow[(v, u)] -= delta

                    # Subtract delta from the supply and demand values of source and target_demand
                    supply[source] -= delta
                    demand[target_demand] -= delta

                    # Remove visited demand nodes if their value is 0
                    demand = {node: demand[node] for node in demand if demand[node] > 0}

        # Update the potential of the nodes based on the distance labels
        for node in nodes:  
                potential[node] = potential[node] - dist[node] 

        # Update the reduced cost of each edge in the residual graph based on the node potentials
        for u, v_dict in residual_graph.items():
            for v, edge_dict in v_dict.items():
                edge_dict['reduced_cost'] = edge_dict['cost'] - potential[u] + potential[v]

        # Remove visited supply nodes if their value is 0
        supply = {node: supply[node] for node in supply if supply[node] > 0}

    # Remove arcs with zero and negative flow from the flow dictionary
    flow = {arc: flow_value for arc, flow_value in flow.items() if flow_value > 0}

    # Calculate the final cost
    final_cost = sum(flow_value * costs[arc] for arc, flow_value in flow.items())

    # Print the flow on each arc
    for arc, flow_value in flow.items():
        print(f"Flow on arc {arc}: {flow_value}")

    # Print the final cost
    print("Optimal Cost:", final_cost)

    # Print Residual Graph
    pprint.pprint(residual_graph)

    return flow

# Dijkstra
    Finding the shortest path from a supply node to other nodes

In [7]:
def dijkstra(graph, nodes, source):
    """
    The Dijkstra's algorithm to find the shortest paths from the source node to all nodes in the graph.

    Arguments:
        graph (dict): Dictionary representing the graph (residual graph)
        nodes (list): List of nodes in the graph
        source: The source node

    Returns:
        dist: A Dictionary containing the distances of nodes from the source
        prev: A Dictionary containing the predecessors of nodes in the shortest paths
    """

    dist = {node: math.inf for node in nodes}  # Initialize distances to infinity for all nodes
    prev = {node: None for node in nodes}  # Initialize predecessor to None for all nodes
    visited_dict = {node: False for node in nodes}  # Initialize visited dictionary with False for all nodes

    dist[source] = 0  # Set the distance of the source node to 0

    while not all(visited for visited in visited_dict.values()):
        # Find the unvisited node (temporarily labeled) with the minimum distance
        u = min((node for node, visited in visited_dict.items() if not visited), key=dist.get)

        visited_dict[u] = True  # Make the node with the minimum distance label permanent

        """
        Update the distance labels and predecessor of adjacent nodes 
        if there is residual capacity (i.e., the edge exists),
        and the adjacent node's distance label satisfies the update condition.
        """
        for v, edge in graph[u].items():
            if edge['capacity'] > 0 and dist[u] + edge['reduced_cost'] < dist[v]:
                dist[v] = dist[u] + edge['reduced_cost']
                prev[v] = u

    return dist, prev

# Example 1

In [8]:
nodes = ['1', '2', '3', '4', '5', '6']
arcs = [('1', '2'), ('1', '3'), ('2', '3'), ('3', '6'), ('3', '5'), ('3', '4'), ('4', '5')]
capacities = {('1', '2'): 3, ('1', '3'): 3, ('2', '3'): 7, ('3', '6'): 1, ('3', '5'): 7, ('3', '4'): 5, ('4', '5'): 3}
costs = {('1', '2'): 1, ('1', '3'): 4, ('2', '3'): 2, ('3', '6'): 8, ('3', '5'): 5, ('3', '4'): 2, ('4', '5'): 1}
supply = {'1': 5, '2': 2}
demand = {'4': 2, '5': 4, '6': 1}

flow = successive_shortest_path(nodes, arcs, capacities, costs, supply, demand)

Flow on arc ('1', '2'): 3
Flow on arc ('1', '3'): 2
Flow on arc ('2', '3'): 5
Flow on arc ('3', '6'): 1
Flow on arc ('3', '5'): 1
Flow on arc ('3', '4'): 5
Flow on arc ('4', '5'): 3
Optimal Cost: 47
{'1': {'2': {'capacity': 0, 'cost': 1, 'reduced_cost': -1},
       '3': {'capacity': 1, 'cost': 4, 'reduced_cost': 0}},
 '2': {'1': {'capacity': 3, 'cost': -1, 'reduced_cost': 1},
       '3': {'capacity': 2, 'cost': 2, 'reduced_cost': 0}},
 '3': {'1': {'capacity': 2, 'cost': -4, 'reduced_cost': 0},
       '2': {'capacity': 5, 'cost': -2, 'reduced_cost': 0},
       '4': {'capacity': 0, 'cost': 2, 'reduced_cost': -2},
       '5': {'capacity': 6, 'cost': 5, 'reduced_cost': 0},
       '6': {'capacity': 0, 'cost': 8, 'reduced_cost': 0}},
 '4': {'3': {'capacity': 5, 'cost': -2, 'reduced_cost': 2},
       '5': {'capacity': 0, 'cost': 1, 'reduced_cost': 0}},
 '5': {'3': {'capacity': 1, 'cost': -5, 'reduced_cost': 0},
       '4': {'capacity': 3, 'cost': -1, 'reduced_cost': 0}},
 '6': {'3': {'capacit

# Example 2

In [9]:
nodes = ['1', '2', '3','4']
arcs = [('1', '2'), ('1', '3'), ('2', '3'),('2','4'), ('3', '4')]
capacities = {('1', '2'): 4, ('1', '3'): 2, ('2', '3'): 2,('2','4'): 3, ('3','4'): 5}
costs = {('1', '2'): 2, ('1', '3'): 2, ('2', '3'): 1, ('2', '4'): 3, ('3','4'): 1}
supply = {'1': 4}
demand = {'4': 4}

flow = successive_shortest_path(nodes, arcs, capacities, costs, supply, demand)

Flow on arc ('1', '2'): 2
Flow on arc ('1', '3'): 2
Flow on arc ('2', '3'): 2
Flow on arc ('3', '4'): 4
Optimal Cost: 14
{'1': {'2': {'capacity': 2, 'cost': 2, 'reduced_cost': 0},
       '3': {'capacity': 0, 'cost': 2, 'reduced_cost': -1}},
 '2': {'1': {'capacity': 2, 'cost': -2, 'reduced_cost': 0},
       '3': {'capacity': 0, 'cost': 1, 'reduced_cost': 0},
       '4': {'capacity': 3, 'cost': 3, 'reduced_cost': 1}},
 '3': {'1': {'capacity': 2, 'cost': -2, 'reduced_cost': 1},
       '2': {'capacity': 2, 'cost': -1, 'reduced_cost': 0},
       '4': {'capacity': 1, 'cost': 1, 'reduced_cost': 0}},
 '4': {'2': {'capacity': 0, 'cost': -3, 'reduced_cost': -1},
       '3': {'capacity': 4, 'cost': -1, 'reduced_cost': 0}}}


# Example 3

In [10]:
nodes = ['1', '2', '3','4','5']
arcs = [('1', '2'), ('1', '3'), ('2', '3'),('2','4'),('3','4'),('3','5')]
capacities = {('1', '2'): 7, ('1', '3'): 7, ('2', '3'): 2,('2','4'):2, ('3','4'):3, ('3','5'):2}
costs = {('1', '2'): 1, ('1', '3'): 5, ('2', '3'): -2,('2','4'):8, ('3','4'):-3, ('3','5'):4}
supply = {'1': 5}
demand = {'4': 3,'5':2}

flow = successive_shortest_path(nodes, arcs, capacities, costs, supply, demand)

Flow on arc ('1', '2'): 2
Flow on arc ('1', '3'): 3
Flow on arc ('2', '3'): 2
Flow on arc ('3', '4'): 3
Flow on arc ('3', '5'): 2
Optimal Cost: 12
{'1': {'2': {'capacity': 5, 'cost': 1, 'reduced_cost': 0},
       '3': {'capacity': 4, 'cost': 5, 'reduced_cost': 0}},
 '2': {'1': {'capacity': 2, 'cost': -1, 'reduced_cost': 0},
       '3': {'capacity': 0, 'cost': -2, 'reduced_cost': -6},
       '4': {'capacity': 2, 'cost': 8, 'reduced_cost': 7}},
 '3': {'1': {'capacity': 3, 'cost': -5, 'reduced_cost': 0},
       '2': {'capacity': 2, 'cost': 2, 'reduced_cost': 6},
       '4': {'capacity': 0, 'cost': -3, 'reduced_cost': 0},
       '5': {'capacity': 0, 'cost': 4, 'reduced_cost': -inf}},
 '4': {'2': {'capacity': 0, 'cost': -8, 'reduced_cost': -7},
       '3': {'capacity': 3, 'cost': 3, 'reduced_cost': 0}},
 '5': {'3': {'capacity': 2, 'cost': -4, 'reduced_cost': inf}}}
